In [3]:
import torch

In [2]:

torch.manual_seed(3)
batch_size = 10000
x = torch.rand(batch_size, 8, 1)
w_base = torch.rand(1, 8, 16)



In [2]:

def calc_chosen(x, w):

    inner = torch.min(x, w)
    y = torch.max(inner, keepdim=True, dim=-2)[0]
    chosen = (y == inner)
    return y, chosen


y_base, chosen_base = calc_chosen(x, w_base)

NameError: name 'x' is not defined

In [59]:
def update_rel(x, t, chosen=None, dim=0):

    negatives = torch.relu(x - t)
    positives = torch.min(x, t)
    if chosen is not None:
        positives = chosen * positives

    return (
        positives.sum(dim=dim, keepdim=True) / 
        (positives.sum(dim=dim, keepdim=True) + negatives.sum(dim=dim, keepdim=True))
    )


In [60]:

# 

chosen_rel = None
for i in range(5):
    w_rel = update_rel(x, y_base, chosen_rel)
    y_rel, chosen_rel = calc_chosen(x, w_rel)

    true_count = (
        (chosen_rel == True) & (chosen_base == True)
    ).float()
    fp_count = (
        (chosen_rel == False) & (chosen_base == True)
    ).float()
    print(
        'Rate: ',
        true_count.sum() /
        (fp_count + true_count).sum()
    )
    print(
        'Loss: ', (y_rel - y_base).abs().mean()
    )
# for the first round i 

# increases to roughly 80% accuracy on the second iteration
# indicates it should help to do an extra iteration

# on the first round do not use chosen

# inner_rel = torch.min(x, w_rel)
# y_rel = inner_rel.max(dim=-2, keepdim=True)
# chosen = (y_rel == inner_rel)

# how many of the indices match

Rate:  tensor(0.4144)
Loss:  tensor(0.1646)
Rate:  tensor(0.8280)
Loss:  tensor(0.0387)
Rate:  tensor(0.8893)
Loss:  tensor(0.0395)
Rate:  tensor(0.8663)
Loss:  tensor(0.0463)
Rate:  tensor(0.8466)
Loss:  tensor(0.0504)


Looks like it is worth it to use rel..

1. Don't know how much it will help in a realistic case but it looks quite promising here
2. It's possible the straight through estimator is also doing as good as it can for multiple layers and that I need to add noise
3. When there is not a clear optimum. Maybe it does not matter too much
4. The loss decreases drastically on the second iteration. Regular gradient descent might get me the rest of the way
5. Not sure how things will be affected by using both x_rel and w_rel. But doing just one seems to help dramatically


For now... Implement a simpler version of the "second order version"


a. Use rel loss 2
b. gradually anneal it
c. get rid of it

d. How to deal with the regular intersection / Union?
 a. Use STE (?)
 b. Do not use it 
 c. Use the "soft version"

In [4]:
from rel import train, MinMax, MinMaxLoss, MinMaxLearner, TruncatedGaussianNoise, MaxMinLearner, MaxMin, RandNoise

import torch.nn as nn
import torch.utils.data
import numpy as np
import zenkai

torch.manual_seed(3)
batch_size = 10000
x = torch.rand(batch_size, 8)

x_weight = 1.0
w_weight = 0.01
n_epochs = 400

neuron_base = MinMax(
    8, 16
)
neuron_base2 = MinMax(
    16, 16
)
neuron1 = MinMaxLearner(
    8, 16, reduction='mean', rel_reduction='mean', 
    x_weight=0.1, w_weight=0.01, a=8
)
neuron2 = MinMaxLearner(
    16, 16, reduction='mean', rel_reduction='mean', 
    x_weight=0.01, w_weight=0.01, a=8
)
noiser = nn.Dropout(0.2)
with torch.no_grad():
    t = neuron_base2(neuron_base(x))

dataset = torch.utils.data.TensorDataset(
    x, t
)
optim = torch.optim.Adam(neuron1.parameters(), lr=1e-3)

j = 0
for i in range(n_epochs):
    results = []
    for x_i, t_i in torch.utils.data.DataLoader(
        dataset, 128, True
    ):
        y_i = neuron2(neuron1(x_i))
        # loss = min_max_loss(x_i, y_i, t_i)

        loss = (y_i - t_i).pow(2).sum()
        # print(
        #     loss.item(),
        #     (y_i - t_i).pow(2).sum() / y_i.numel()
        # )
        optim.zero_grad()
        loss.backward()
        optim.step()
        zenkai.params.apply_p(
            neuron1, lambda p: torch.clamp(p, 0.0, 1.0) 
        )
        zenkai.params.apply_p(
            neuron2, lambda p: torch.clamp(p, 0.0, 1.0) 
        )
        # results.append(loss.item())
        results.append(loss.item() / y_i.numel())
    
    # if i % 2 == 0:
    # neuron1.w_weight = neuron1.w_weight * 0.95
    # neuron2.w_weight = neuron2.w_weight * 0.95
    # neuron2.x_weight = neuron2.x_weight * 0.95
    # neuron1.a = neuron1.a + 1
    #neuron2.a = neuron2.a + 1
    # else:
    #     neuron1.w_weight = neuron1.w_weight * 0.95
    #     neuron2.w_weight = 0.0
    #     neuron2.x_weight = 0.1

    print(np.mean(results))



0.009311253794386416
0.009936544737672503
0.013782449302416814
0.019162020598880097
0.024298094189431095
0.02743134305729896
0.03035657768008075
0.03272563341674926
0.03546803260717211
0.03723054567847071
0.03830222078139269
0.04047237855346897


KeyboardInterrupt: 